# A Generative Approach to Modeling Emergent Threats in Multi Agentic Systems

In [4]:
# === Cell 2: Config & Runtime Switches =======================================
# Demo mode controls for Approach 2 (with a fallback to Approach 1)
# Notes:
# - LLM = Large Language Model (expanded once per your preference)
# - We’re using only GPT models for now.
# - Temperature is phase-aware: more creative for ideation, lower for evaluation.

from pathlib import Path
import os, random
import numpy as np

# ---- Approach toggle ---------------------------------------------------------
APPROACH = "A2"        # "A2" = full (graph + drift), "A1" = minimalist fallback

# ---- Execution toggles -------------------------------------------------------
USE_CACHED_OUTPUTS = True   # Load/save agent outputs to disk (good for offline demos)
LIVE_LLM = False            # If False, only uses cached outputs (safe on bad Wi-Fi)

# ---- Models (GPT only) -------------------------------------------------------
# Keep them both GPT-family for now; you can switch to a larger judge later if needed.
PRIMARY_MODEL = "gpt-4o-mini"   # used for vulnerability agents + SCAMPER ideation
EVAL_MODEL    = "gpt-4o"        # used for evaluation/consensus (still GPT)

# ---- Temperatures by phase ---------------------------------------------------
# Lower = more deterministic; Higher = more creative.
TEMPS = {
    "temporal_vuln": 0.3,
    "interaction_vuln": 0.3,
    "behavioral_vuln": 0.3,
    "scamper_threats": 0.6,   # a bit more creativity for story-like threats
    "evaluation": 0.2,        # stricter scoring / less variance
}

def temp_for(phase: str) -> float:
    return TEMPS.get(phase, 0.3)

# ---- Reproducibility ---------------------------------------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

# ---- Paths (cache + outputs) -------------------------------------------------
BASE_DIR = Path().resolve()  # current notebook folder
CACHE_DIR = BASE_DIR / ".cache"
OUT_DIR   = BASE_DIR / "artifacts"
CACHE_DIR.mkdir(exist_ok=True)
OUT_DIR.mkdir(exist_ok=True)

# ---- Helper: show current config --------------------------------------------
def print_config():
    print("=== Demo Config ===")
    print(f"Approach:             {APPROACH}  (A2=full, A1=fallback)")
    print(f"Use cached outputs:   {USE_CACHED_OUTPUTS}")
    print(f"Live LLM calls:       {LIVE_LLM}")
    print(f"Primary model:        {PRIMARY_MODEL}")
    print(f"Evaluation model:     {EVAL_MODEL}")
    print(f"Temperatures:         {TEMPS}")
    print(f"Seed:                 {SEED}")
    print(f"Cache dir:            {CACHE_DIR}")
    print(f"Artifacts dir:        {OUT_DIR}")

print_config()
# ============================================================================ #


=== Demo Config ===
Approach:             A2  (A2=full, A1=fallback)
Use cached outputs:   True
Live LLM calls:       False
Primary model:        gpt-4o-mini
Evaluation model:     gpt-4o
Temperatures:         {'temporal_vuln': 0.3, 'interaction_vuln': 0.3, 'behavioral_vuln': 0.3, 'scamper_threats': 0.6, 'evaluation': 0.2}
Seed:                 42
Cache dir:            /Users/vikramnarayan/Documents/ERIMAS/TMC-Washington-Nov-2025/.cache
Artifacts dir:        /Users/vikramnarayan/Documents/ERIMAS/TMC-Washington-Nov-2025/artifacts


In [6]:
# === Cell 2.1: Load Environment Variables and Initialize OpenAI ===============

from dotenv import load_dotenv
import openai
import os

# Load variables from .env file in the project root
load_dotenv()

# Check for key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise EnvironmentError("❌ OPENAI_API_KEY not found. Please create a .env file in the project root.")

# Initialize client
openai.api_key = api_key

# (Optional) small confirmation print, without leaking the key
print("✅ OpenAI client initialized.")
print(f"Key prefix: {api_key[:8]}... (hidden)")


✅ OpenAI client initialized.
Key prefix: sk-proj-... (hidden)
